In [2]:
import numpy as np
import pandas as pd
import scipy
import os
#---------------------Peleng fucntions ----------------------------------------------------------------------
def SetCSFunc(CSType):
    def GetDC(TX_R, TX_Az, TX_El):
        VDC = np.zeros(3)
        if CSType == 'AoE':
            VDC[0] = np.sin(TX_Az) * np.cos(TX_El)
            VDC[1] = np.sin(TX_El)
            VDC[2] = np.cos(TX_Az) * np.cos(TX_El)
        elif CSType == 'Z-AoE':
            VDC[0] = np.sin(TX_Az) * np.cos(TX_El) * (np.cos(TX_Az) * np.cos(TX_El)) ** (-1)
            VDC[1] = np.sin(TX_El) * (np.cos(TX_Az) * np.cos(TX_El)) ** (-1)
            VDC[2] = 1
        return VDC * TX_R
    return GetDC

def SetMRotate(n, Plane, angle):
    i = Plane[0]
    j = Plane[1]
    Res = np.identity(n)
    Res[i][j] = np.cos(angle)
    Res[i][j] = ((-1)**(i+j)) * np.sin(angle)
    Res[j][i] = - Res[i][j]
    Res[j][j] =  Res[i][i]
    return Res

def FSetOffset(TX_R, TX_Az, TX_El, VRX, IsPlaneWave, CSType):
    if IsPlaneWave == 1:
        return (TX_R, TX_Az, TX_El)
    else:
        FCS = SetCSFunc(CSType)
        VDC = FCS(1,TX_Az, TX_El)
        rho = mag(VDC)
        Dist = mag(TX_R * VDC - VRX)
        VDC[0] = VDC[0] * rho * TX_R / Dist
        VDC[1] = VDC[1] * rho * TX_R / Dist
        VDC[2] = cmag(np.array([rho, 1j * VDC[0], 1j*VDC[1]]))
        Az, El = GetAngles(CSType, VDC)
        return Dist, Az, El
    
def SetAPD(TX_R, TX_Az, TX_El, RX_Roll, VRX, F0, IsPlaneWave, CSType, FSetOffset):
    #CSType = lookup(CSType, VCSTypes, VCSRTypes) if IsPlaneWave == 1
    if IsPlaneWave == 1:
        FCS = SetCSFunc(CSType)
        VDC = FCS(1, TX_Az, TX_El)
        Buf = 0.5 * np.sqrt(2) *  SetMRotate(2,np.array([0,1]), -RX_Roll) @ (np.array([VDC[0],VDC[1]]))
        VDC[0], VDC[1], VDC[2] = Buf[0], Buf[1], 0
        Dist = mag(VRX) * np.sign(VRX) @ VDC
    else:
        VRX = np.matmul(SetMRotate(3, np.array([0,1]), RX_Roll) , VRX)
        Dist, _, _ = FSetOffset(TX_R, TX_Az, TX_El, VRX, IsPlaneWave, CSType)
    A = 1* FPlu(Dist + (1e3 if IsPlaneWave == 1 else 0), F0)
    P = GetAngle(((-1) ** (IsPlaneWave+1)) * Dist * GetK(F0))
    return A,P

def GetAngles(SType, VDC):
    R = cmag(VDC)
    U = np.real(VDC[0])
    V = np.real(VDC[1])
    W = np.real(VDC[2])
    if SType == 'AoE':
        try:
            Az = np.arcsin(U) if IsNaN(R) == 1 else np.arctan2(U, W)
            El = np.arcsin(V) if IsNaN(R) == 1 else np.arcsin(V / R)
        except:
            Az = None
            El = None
    if SType == 'Z-AoE':
        try:
            Az = np.arcsin(U) if IsNaN(R) == 1 else np.arctan2(U,W)
            El = np.arcsin(V) if IsNaN(R) == 1 else np.arcsin(V/R)
        except:
            Az = None
            El = None
    return np.real(Az), np.real(El)

def dB2VR(dB):
    return 10**(dB/20)

def lookup(z, A, B):
    i = 0
    while z != A[i]:
        i = i + 1
    return B[i]

def IsNaN(a):
    return 1 if a == None else 0

def cmag(V):
    return np.sqrt(np.matmul(V.transpose(), V))

def mag(V):
    return np.sqrt(np.matmul(V.transpose(), np.conjugate(V)))

def FPlu(D, F0):
    return (2 * D * GetK(F0)) ** (-1)

def GetK(F):
    return 2 * np.pi * (1e8 * 2.998 / F) ** (-1)

def SetSolver(Base,TX_R,RX_Z,delta,IsPlaneWave,IsRegularCS):
    #СК РЕГ
    IsRegularCS = np.max([IsRegularCS, IsPlaneWave])
    def GetF(DC,delta,Index,Sign):
        if IsPlaneWave == 0:
            Res = ((TX_R ** 2) * (Base ** 2) * (DC[1-Index] ** 2))  - delta ** 2 * (TX_R ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R * DC[2] + 0.5 * Base ** 2 + (-1) ** Sign * Base * TX_R * DC[Index] - 1/4 * delta ** 2)
        elif IsPlaneWave == 1:
            Res = delta + (-1) ** ((Index + Sign) % 2)  * Base * DC[1-Index]
        return Res

    def GetDeltaX(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = (-1) ** (Sign + 1) * Base * TX_R * delta ** 2
        else:
            Res = 0 # Заглушка
        return Res

    def GetDeltaY(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = 2 * Base ** 2 * TX_R ** 2 * DC[1 - Index]
        else:
            Res = (-1) ** ((Index + Sign) % 2) * Base # Заглушка
        return Res

    def GetDeltaW(DC, delta, Index, Sign):
        if IsPlaneWave == 0:
            Res = 2 * RX_Z * TX_R * delta ** 2
        else:
            Res = 0 # Заглушка
        return Res

    # Заполняем Массива Градиента И Функции СК РЕГ
    def grad_RCS(DC):
        Res = np.empty((5,3), dtype = complex)
        Res[0][0] = GetDeltaX(DC, delta[3], 0, 0)
        Res[0][1] = GetDeltaY(DC, delta[3], 0, 0)
        Res[0][2] = GetDeltaW(DC, delta[3], 0, 0)

        Res[1][0] = GetDeltaY(DC, delta[0], 1, 0)
        Res[1][1] = GetDeltaX(DC, delta[0], 1, 0)
        Res[1][2] = GetDeltaW(DC, delta[0], 1, 0)

        Res[2][0] = GetDeltaX(DC, delta[1], 0, 1)
        Res[2][1] = GetDeltaY(DC, delta[1], 0, 1)
        Res[2][2] = GetDeltaW(DC, delta[1], 0, 1)

        Res[3][0] = GetDeltaY(DC, delta[2], 1, 1)
        Res[3][1] = GetDeltaX(DC, delta[2], 1, 1)
        Res[3][2] = GetDeltaW(DC, delta[2], 1, 1)

        Res[4][0] = 2 * DC[0]
        Res[4][1] = 2 * DC[1]
        Res[4][2] = 2 * DC[2]
        return Res

    def Func_RCS(DC):
        Res = np.empty((5), dtype = complex)
        Res[0] = GetF(DC, delta[3], 0,0)
        Res[1] = GetF(DC, delta[0], 1,0)
        Res[2] = GetF(DC, delta[1], 0, 1)
        Res[3] = GetF(DC, delta[2], 1, 1)
        Res[4] = cmag(DC) ** 2 -1
        return Res

    # СК НЕ РЕГ

    def GetF1(DC, delta, Index, Sign):
        Res = Base ** 2 * TX_R ** 2 * (DC[1-Index]) ** 2 - (DC[2]) ** 2 * delta ** 2 * (RX_Z ** 2 - 2 * RX_Z * TX_R + 0.5 * Base ** 2 - 0.25 * delta ** 2) - (TX_R ** 2 + (-1) ** Sign * Base * TX_R * DC[Index] * DC[2]) * delta ** 2
        return Res

    def GetDeltaX1(DC, delta, Index, Sign):
        Res = 2 * DC[Index] * delta ** 2 * (0.5 * Base ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R - 1/4 * delta ** 2) - (-1) ** Sign * Base * TX_R * DC[2] * delta ** 2
        return Res

    def GetDeltaY1(DC, delta, Index, Sign):
        Res = 2 * DC[1 - Index] * delta ** 2 * (0.5 * Base ** 2 + RX_Z ** 2 - 2 * RX_Z * TX_R - 1/4 * delta ** 2) + 2 * Base ** 2 * TX_R ** 2 * DC[1 - Index]
        return Res

    def GetDeltaW1(DC, delta, Index, Sign):
        Res = (-1) ** (Sign + 1) * Base * TX_R * DC[Index] * delta ** 2
        return Res

    def Func_ZCS(DC):
        Res = np.empty((5), dtype = complex)
        Res[0] = GetF1(DC, delta[3], 0,0)
        Res[1] = GetF1(DC, delta[0], 1,0)
        Res[2] = GetF1(DC, delta[1], 0, 1)
        Res[3] = GetF1(DC, delta[2], 1, 1)
        Res[4] = cmag(DC) -1
        return Res


    def grad_ZCS(DC):
        Res = np.empty((5,3), dtype = complex)

        Res[0][0] = GetDeltaX1(DC, delta[3], 0, 0)
        Res[0][1] = GetDeltaY1(DC, delta[3], 0, 0)
        Res[0][2] = GetDeltaW1(DC, delta[3], 0, 0)

        Res[1][0] = GetDeltaY1(DC, delta[0], 1, 0)
        Res[1][1] = GetDeltaX1(DC, delta[0], 1, 0)
        Res[1][2] = GetDeltaW1(DC, delta[0], 1, 0)

        Res[2][0] = GetDeltaX1(DC, delta[1], 0, 1)
        Res[2][1] = GetDeltaY1(DC, delta[1], 0, 1)
        Res[2][2] = GetDeltaW1(DC, delta[1], 0, 1)

        Res[3][0] = GetDeltaY1(DC, delta[2], 1, 1)
        Res[3][1] = GetDeltaX1(DC, delta[2], 1, 1)
        Res[3][2] = GetDeltaW1(DC, delta[2], 1, 1)

        Res[4][0] = 2 * DC[0]
        Res[4][1] = 2 * DC[1]
        Res[4][2] = 2 * DC[2]
        return Res
    return (Func_ZCS, grad_ZCS) if IsRegularCS == 0  else (Func_RCS, grad_RCS)

def DirFind(Vdx, Base, RX_Z, RX_Roll, TX_R, CSType, IsPlaneWave, N, eps):
    VCSTypes = ['Z-AoE', 'AoE', 'PS']
    VCSRTypes = ['AoE', 'AoE', 'PS']
    CSRType = lookup(CSType, VCSTypes, VCSRTypes)
    IsRegularCS = (CSType == CSRType)
    Func, FGrad = SetSolver(Base,TX_R,RX_Z,Vdx,IsPlaneWave,IsRegularCS)
    VDC0 = np.stack([0.5,0.5])
    VDC0 = np.append(VDC0, cmag(np.append(1j*VDC0,1)))
    WD_Max = 25
    WD = 0
    i = 0
    err = np.array([eps + 1])
    VDC = VDC0
    VDCN = np.empty(3,dtype=complex)
    while i<N if N > 0 else mag(err) > eps:
        J = FGrad(VDC)
        try:

            VDCN = VDC - np.matmul(np.matmul(np.linalg.matrix_power(np.matmul(J.transpose(),J),-1), J.transpose()) ,Func(VDC))
            #VDCN = VDC - np.matmul(J.transpose(), J) ** (-1), J.transpose(), Func(VDC)
        except:
            WD = WD + 1
            i = N if WD > WD_Max else 0
            VDCN[2] = cmag(np.append(1j*VDCN, 1)) if WD <= WD_Max else None
            VDCN[0] = np.random.rand(1)  if WD<=WD_Max else None
            VDCN[1] = np.random.rand(1)  if WD<=WD_Max else None
        err = VDCN - VDC
        VDC = VDCN
        i = i + 1
    VDC[0] = VDC[0] * (1 if IsPlaneWave == 1 else np.sign(Vdx[0]))
    VDC[1] = VDC[1] * (1 if IsPlaneWave == 1 else np.sign(Vdx[1]))
    VDC = np.matmul(SetMRotate(3, np.stack([0,1]), RX_Roll) , VDC)
    if IsPlaneWave == 1:
        VDC[2] = cmag(np.stack([1, 1j*VDC[0], 1j*VDC[1]],1j*VDC[2]))
    VDC[2] = cmag(np.stack([1, 1j*VDC[0], 1j*VDC[1]]))
    return GetAngles(CSType, VDC)
#-------------------------------------------------------------------------------------------------

def PR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 10 * np.log10(val)

def VR2dB(val):         # 2Do: add array support - REWRITE LATER
    return 2 * PR2dB(val)

def GetAngle(alpha):
    return np.arctan2(np.sin(alpha), np.cos(alpha))

def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])

def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F

def read_data(file_dict : dict):
    absolute_path = os.path.join(file_dict['CD'],file_dict['file_name'])
    names = [['TimeStamp'],['Freq'],['Azimuth'],['Tilt']]
    element_MA = [['A{}_MAG'.format(i),'A{}_ARG'.format(i)] for i in range(4)]
    names.extend(element_MA)
    names = [item for element in names for item in element]
    data_frame = pd.read_csv(absolute_path,sep=';',comment = '#',header = None,names=names)
    return data_frame

def filter_data(df,freq,az,tilt,num_of_element,iloc):
    locls = locals()
    errors = 0
    for e in ['df', 'iloc', 'num_of_element']:
        locls.pop(e)
    for key in locls.keys():
        if locls[key] not in df.values:
            print(f'{key} = {locls[key]} not in DataFrame')
            errors = 1
    if errors == 1:
        return 0
    return df[((df.Freq == freq) & (df.Azimuth == az) & (df.Tilt == tilt))][['A{}_MAG'.format(num_of_element),'A{}_ARG'.format(num_of_element)]].iloc[iloc].to_numpy()

def convert_data(src,src_format, dst_format):
    if src_format == dst_format:
        return src
    else:
        src_mag_format, src_arg_format = src_format.split('|')
        dst_mag_format, dst_arg_format = dst_format.split('|')
        if src_mag_format != dst_mag_format:
            if src_mag_format == 'dB' and dst_mag_format == 'VR':
                src[0] = 10**(0.1*src[0])
            else:
                src[0] = PR2dB(src[0])
        if src_arg_format != dst_arg_format:
            if src_arg_format == 'deg' and dst_arg_format == 'rad':
                src[1] = np.deg2rad(src[1])
            else:
                src[1] = np.rad2deg(src[1])
        return src
    
def set_beat(VSA: np.ndarray, VSB : np.ndarray, return_type = 'dB|deg') -> np.ndarray:
    ampl = np.sqrt(VSA[0]**2 + VSB[0]**2 + 2* VSA[0] * VSB[0] * np.cos(VSB[1] - VSA[1]))
    phase = np.arctan2((VSA[0] * np.sin(VSA[1]) + VSB[0] * np.sin(VSB[1])),(VSA[0] * np.cos(VSA[1]) + VSB[0] * np.cos(VSB[1])))
    type_ampl, type_phase = return_type.split('|')
    if type_ampl == 'dB':
        ampl = PR2dB(ampl)
    if type_phase == 'deg':
        phase = np.rad2deg(phase)
    return np.array([ampl,phase,])

def beat_function(S12,SRC,VAP):
    F = np.zeros(2)
    F[0] = -S12[0]**2 + SRC[0]**2 + VAP[0]**2 + 2 * SRC[0] * VAP[0] * np.cos(SRC[1] - VAP[1])
    F[1] = -np.tan(S12[1]) + (SRC[0]*np.sin(SRC[1]) + VAP[0]*np.sin(VAP[1]))/ (SRC[0]*np.cos(SRC[1]) + VAP[0] * np.cos(VAP[1]))
    return F

# \\\ Macros section

In [22]:
df = read_data(dict(CD = 'C:\\Users\\vovan\\PycharmProjects\\GUI_Test\\Meas\\Meas_KTRV', file_name = 'Meas_27.05.23.txt'))
filtered_s12 = filter_data(df,freq = 4e9,az = 15,tilt = 0,num_of_element=3,iloc = 0)
filtered_src = filter_data(df,freq = 4e9,az = 0,tilt = 0,num_of_element=0,iloc = 9)
filtered_s12 = convert_data(filtered_s12,'dB|deg','VR|rad')
filtered_src = convert_data(filtered_src,'dB|deg','VR|rad')
fun = lambda VAP : beat_function(filtered_s12,filtered_src,VAP)
f0 = [filtered_s12[0],0]
ampl, phase  = scipy.optimize.fsolve(fun,f0,maxfev=1000)
interfer = np.array([ampl,GetAngle(phase + np.pi)])
result = set_beat(filtered_s12,interfer)
print(f'S12 ={convert_data(filtered_s12,"VR|rad","dB|deg")}')
print(f'SRC ={convert_data(filtered_src,"VR|rad","dB|deg")}')
print(f'Compensated  = {result}')


S12 =[-43.862382 -43.813626]
SRC =[-44.284054 147.574601]
Compensated  = [-44.284054   147.57460101]


In [272]:
np.random.randn()

2.1164922762851344

In [110]:
deg = lambda val : np.deg2rad(val)
rad = lambda val : np.rad2deg(val)
VSA = [0.0005,deg(0)]
VSB = [0.0002,deg(170)]
VSR = set_beat(VSA,VSB,'VR|rad')
SRC = VSA
S12 = VSR
fun = lambda VAP : beat_function(S12,SRC,VAP)
ampl, phase  = scipy.optimize.fsolve(fun,[0,0])
result = np.array([ampl,GetAngle(phase + np.pi)])
phase = rad(GetAngle(phase))
print(f'Ampl = {ampl}, phase = {phase} deg')
set_beat(S12,result)


Ampl = 4.537253968548578e-08, phase = 5.089498156840467e-07 deg


array([-44.37385596, -38.4860302 ])